In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
import pandas as pd
import numpy as np
import re

from bag_of_words import *
from regex_predictor import *
from odds_ratio import *
from proximal import *

In [2]:
master_data = pd.read_csv('csv_files/master_data.csv')

### Bag of words linear regression to see if text data carries information about kidney failure.

In [3]:
text_clf = create_bow_model(max_words=25000, ngram_range=(1, 2))

text_clf.fit(master_data['notes_half2'], master_data['kidney_fail'])

/home/jchen/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect',
                 CountVectorizer(max_features=25000, ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('lr', LogisticRegression(C=1e+20))])

In [4]:
print('majority class', 1-np.mean(master_data['kidney_fail']))

majority class 0.7958303622966962


In [4]:
predicted = text_clf.predict(master_data['notes_half2'])

print(np.mean(predicted == master_data['kidney_fail']))

0.8723982207734882


In [49]:
create_confusion_matrix(master_data['kidney_fail'], predicted)

{'tn': 22154,
 'fp': 1284,
 'fn': 2443,
 'tp': 3570,
 'sensitivity': 0.5937136204889406,
 'specificity': 0.9452171687004011,
 'precision': 0.7354758961681088,
 'recall': 0.5937136204889406}

In [6]:
coef = text_clf['lr'].coef_[0]
print(coef)

words = text_clf['vect'].get_feature_names_out()
print(words)

sorted({w:c for w, c in zip(words, coef)}.items(), key=lambda kv: -kv[1])[2000:]

[-3.3645458  -1.82385919 -1.2642783  ... -0.42634049  0.83231731
  0.7452317 ]
['00' '00 the' '01' ... 'zofran with' 'zone' 'zosyn']


[('section the', 3.157253451526739),
 ('the technical', 3.156148888128932),
 ('moderate dilatation', 3.1561109399959215),
 ('infusions of', 3.1555390703885893),
 ('70 right', 3.153902979364341),
 ('study the', 3.1536809014820464),
 ('to off', 3.1517717737434814),
 ('no significant', 3.15128954132508),
 ('prior report', 3.151193747502173),
 ('either study', 3.1501495864626636),
 ('lity', 3.1489019612156723),
 ('45 tissue', 3.1481425369778093),
 ('moderate or', 3.1473677638026643),
 ('revealed', 3.1458109367462685),
 ('higher no', 3.145748953603419),
 ('2189 10', 3.14570988274709),
 ('at low', 3.1456035999327674),
 ('team', 3.144816223123768),
 ('at osh', 3.14401688357516),
 ('artery normal', 3.1422269483508734),
 ('level', 3.1416608008400924),
 ('was performed', 3.1412631314131114),
 ('lead ii', 3.1412124407137108),
 ('associated', 3.140216669912822),
 ('chordae and', 3.1399687752280028),
 ('yellowish', 3.1398141775044053),
 ('infusing at', 3.139352410674411),
 ('echo evidence', 3.13900

### Try out regex predictors.

In [125]:
# doppler, *pressure (high blood pressure causes kidney failure), **valve (but may not be correlated enough, has 50% accuracy)
regex_predicted = regular_expression_predict(master_data['notes_half2'], ['pressure'])

print(np.mean(regex_predicted))

print(np.mean(regex_predicted == master_data['kidney_fail']))

create_confusion_matrix(master_data['kidney_fail'], regex_predicted)

0.2072595158059149
0.7135241587721979


{'tn': 19174,
 'fp': 4264,
 'fn': 4173,
 'tp': 1840,
 'sensitivity': 0.30600365873939794,
 'specificity': 0.818073214438092,
 'precision': 0.30144167758846657,
 'recall': 0.30600365873939794}